In [ ]:
import os
import time
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.linalg import svd, norm, qr
from memory_profiler import memory_usage

# --- File paths ---
base_path = r'G:\My Drive\NUS\NUS Y6S1\ME5311\PROJECT_2420_ME5311'
slp_path = os.path.join(base_path, 'slp.nc')
t2m_path = os.path.join(base_path, 't2m.nc')

# --- Load datasets ---
ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

slp = ds_slp['msl'].values
t2m = ds_t2m['t2m'].values
timestamps = ds_slp['time'].values
lats = ds_slp['latitude'].values
longs = ds_slp['longitude'].values

# --- Reshape and center SLP ---
n_time, n_lat, n_lon = slp.shape
A_slp = slp.reshape(n_time, -1).T
A_mean_slp = A_slp.mean(axis=1, keepdims=True)
A_centered_slp = A_slp - A_mean_slp

In [ ]:
# --- Define QR-based SVD function with monitoring ---
def perform_qr_based_svd_with_monitoring(A):
    """Performs QR-based SVD with runtime and memory tracking"""
    print("Performing QR-based Low-Rank SVD on SLP data...")
    
    def qr_svd_task():
        global Q, R, U_R, S_slp, VT_slp, U_slp
        
        # Step 1: QR decomposition of A
        # Use mode='reduced' instead of 'economic' (which is deprecated)
        print("Step 1")
        Q, R = qr(A, mode='reduced')  
        
        # Step 2: SVD of R (smaller matrix)
        print("Step 2")
        U_R, S_slp, VT_slp = svd(R, full_matrices=False)
        
        # Step 3: Compute U = Q*U_R
        print("Step 3")
        U_slp = Q @ U_R
        
        # Note: U_slp is n×m, S_slp is m, VT_slp is m×m

    # Monitor runtime and memory usage
    start = time.time()
    mem_usage = memory_usage(qr_svd_task, max_usage=True)
    elapsed = time.time() - start
    
    return elapsed, mem_usage

# --- Run QR-based SVD with performance monitoring ---
elapsed_slp, peak_mem_slp = perform_qr_based_svd_with_monitoring(A_centered_slp)

# --- SVD result shapes (informational) ---
print(f"A shape: {A_slp.shape}")
print(f"Q shape: {Q.shape}, R shape: {R.shape}")
print(f"U shape: {U_slp.shape}, S shape: {S_slp.shape}, VT shape: {VT_slp.shape}")

In [ ]:
# --- Accuracy (Reconstruction Error) ---
def calculate_reconstruction_error(U, S, VT, A_original, A_mean):
    """Calculate reconstruction error using Frobenius norm"""
    # Create diagonal S matrix for matrix multiplication
    S_diag = np.diag(S)
    
    # Reconstruct the original matrix
    A_reconstructed = U @ S_diag @ VT + A_mean
    
    # Calculate relative error
    error = norm(A_original - A_reconstructed) / norm(A_original)
    return error, A_reconstructed

# --- Noise Robustness Test ---
def test_noise_robustness(A_centered, A_original, A_mean, noise_scale=0.01):
    """Test QR-SVD robustness against Gaussian noise"""
    np.random.seed(0)  # For reproducibility
    noise = np.random.normal(scale=noise_scale, size=A_centered.shape)
    A_noisy = A_centered + noise
    
    # Perform QR-SVD on noisy data
    Q_noisy, R_noisy = qr(A_noisy, mode='reduced')
    U_R_noisy, S_noisy, VT_noisy = svd(R_noisy, full_matrices=False)
    U_noisy = Q_noisy @ U_R_noisy
    
    # Calculate reconstruction error with noise
    S_noisy_diag = np.diag(S_noisy)
    A_reconstructed_noisy = U_noisy @ S_noisy_diag @ VT_noisy + A_mean
    
    error = norm(A_original - A_reconstructed_noisy) / norm(A_original)
    return error

# Calculate reconstruction error
reconstruction_error, A_reconstructed = calculate_reconstruction_error(
    U_slp, S_slp, VT_slp, A_slp, A_mean_slp
)

# Test noise robustness
noise_error = test_noise_robustness(A_centered_slp, A_slp, A_mean_slp)

# --- Report results ---
print("\n===== QR-based SVD Results for SLP =====")
print(f"Runtime: {elapsed_slp:.2f} seconds")
print(f"Peak memory usage: {peak_mem_slp:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error:.6e}")



In [ ]:
# --- Optional: Save results to file for later comparison ---
# Convert NumPy values to Python native types
results = {
    "method": "QR-based SVD",
    "runtime": float(elapsed_slp),
    "memory_usage": float(peak_mem_slp),
    "reconstruction_error": float(reconstruction_error),
    "noise_robustness": float(noise_error),
    "top_singular_values": [float(sv) for sv in S_slp[:10]]  # Convert each singular value to Python float
}

# Save as JSON (optional)
import json
with open("qr_svd_results.json", "w") as f:
    json.dump(results, f, indent=4)

# --- Optional: Plot singular value decay ---
plt.figure(figsize=(10, 6))
plt.semilogy(range(1, min(101, len(S_slp))), S_slp[:100], 'o-')
plt.title('Singular Value Decay (Top 100) - QR-based SVD')
plt.xlabel('Index')
plt.ylabel('Singular Value (log scale)')
plt.grid(True)
plt.savefig('qr_svd_singular_values.png', dpi=300)
plt.show()